In [ ]:

import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
import numpy as np
from keras import optimizers
from keras.callbacks import ModelCheckpoint
import cv2
from tqdm import tqdm
import random
import os
from sklearn.model_selection import train_test_split 



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def prepare_datasets(validation_size, X, y):
    """Loads data and splits it into train, validation and test sets.
    :param test_size (float): Value in [0, 1] indicating percentage of data set to allocate to test split
    :param validation_size (float): Value in [0, 1] indicating percentage of train set to allocate to validation split
    :return X_train (ndarray): Input training set
    :return X_validation (ndarray): Input validation set
    :return X_test (ndarray): Input test set
    :return y_train (ndarray): Target training set
    :return y_validation (ndarray): Target validation set
    :return y_test (ndarray): Target test set
    """

    # load data
    #X, y = load_data()

    # create train, validation and test split
    X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=validation_size)
    #X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    return X_train, X_validation, y_train, y_validation

In [ ]:
data = []
labels = []
DATADIR = "/content/drive/MyDrive/try1000_model_cough/melspectro_selected/training"

CATEGORIES = ['clapping', 'coughing','crying_baby', 'laughing', 'sneezing']
training_data = []
for category in CATEGORIES:  # do cough and notcough

    path = os.path.join(DATADIR,category)  # create path to cough and notcough
    print(path)
    class_num = CATEGORIES.index(category)  # get the classification  

    for img in tqdm(os.listdir(path)):  # iterate over each image per cough and notcough
        try:
            img_array = cv2.imread(os.path.join(path,img))  # convert to array
            new_array = cv2.resize(img_array, (224, 224))  # resize to normalize data size
            #image = preprocess_input(new_array)
            training_data.append([new_array, class_num])  # add this to our training_data
        except Exception as e:  # in the interest in keeping the output clean...
            pass
        
random.shuffle(training_data)

X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)
X = np.array(X).reshape(-1, 224, 224, 3)
data = np.array(X, dtype="float32")
labels = np.array(y)
print(labels)
# perform one-hot encoding on the labels
#lb = LabelBinarizer()
#labels = lb.fit_transform(labels)
#labels = to_categorical(labels)
def one_hot_encoding(y):
        y_encoding=[]
        #print(y)
        for i in y:
            #print(type(i))
            expected = [0 for i in range(len(CATEGORIES))]
            expected[i] = 1 #one hot encoding
            y_encoding.append(expected)
        return y_encoding
labels=one_hot_encoding(labels)
print(labels)
#(trainX, testX, trainY, testY) = train_test_split(data, labels,
	#test_size=0.20, stratify=labels, random_state=42)
# get train, validation, test splits
trainX, X_validation, trainY, y_validation = prepare_datasets(0.1, data, labels)

  0%|          | 0/32 [00:00<?, ?it/s]

/content/drive/MyDrive/try1000_model_cough/melspectro_selected/training/clapping
/content/drive/MyDrive/try1000_model_cough/melspectro_selected/training/coughing


  0%|          | 0/32 [00:00<?, ?it/s]

/content/drive/MyDrive/try1000_model_cough/melspectro_selected/training/crying_baby
/content/drive/MyDrive/try1000_model_cough/melspectro_selected/training/laughing


100%|██████████| 32/32 [00:00<00:00, 218.31it/s]


/content/drive/MyDrive/try1000_model_cough/melspectro_selected/training/sneezing
[0 1 4 3 2 1 3 0 4 2 1 3 0 1 0 4 0 3 4 3 1 1 1 4 4 1 2 3 3 2 0 1 2 4 4 3 3
 2 2 2 0 2 0 3 4 4 4 1 3 2 1 0 0 3 1 4 3 0 3 4 4 4 4 3 2 2 0 2 4 3 0 1 1 2
 0 1 1 0 1 3 0 2 4 0 3 2 0 0 3 3 3 2 1 4 4 3 1 0 3 0 2 1 1 2 4 3 4 2 2 2 1
 1 0 1 2 2 3 4 4 4 1 1 3 2 0 0 1 4 2 2 3 2 3 2 3 0 1 0 0 1 1 4 0 3 0 2 4 4
 1 3 1 4 2 3 0 2 4 0 4 0]
[[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 0, 0, 1], [1, 0, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1

In [ ]:
batch_size = 20 #40
epochs = 200 #200

# dimensions of our images.
img_width, img_height = 224, 224

input_tensor = Input(shape=(224,224,3))

#nb_training_samples =  2560  # 1600
#nb_validation_samples =  256 # 400 # Set parameter values

n_targets = 5

#%%
# validation generator configuration
#validation_data_dir = 'wavelets_cough_notcough/testing/'
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [ ]:
trainX = np.array(trainX)
trainY = np.array(trainY)
X_validation = np.array(X_validation)
y_validation = np.array(y_validation)

base_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
print('Model loaded.')
base_model.summary()

#%%

top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(n_targets, activation='softmax'))
top_model.summary()


#%%


model = Model(inputs=base_model.input, outputs=top_model(base_model.output))
model.summary()


#%%

num_layers_to_freeze = 15


#%%

for layer in model.layers[:num_layers_to_freeze]:
    layer.trainable = False


model.compile(optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), 
                      loss='categorical_crossentropy', 
                      metrics=['accuracy'])

# serialize model to JSON
model_json = model.to_json()
model_filename = "/content/drive/MyDrive/try1000_model_cough/vgg16_model_{}_frozen_layers.json".format(num_layers_to_freeze)

with open(model_filename, "w") as json_file:
    json_file.write(model_json)
    
    
#%%

filepath = "/content/drive/MyDrive/try1000_model_cough/esc50_vgg16_stft_weights_train_last_2_base_layers_best.hdf5"

best_model_checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [best_model_checkpoint]


model.fit(
    aug.flow(trainX, trainY, batch_size=batch_size),
    steps_per_epoch=len(trainX)//batch_size,
    epochs=epochs,
    validation_data=(X_validation, y_validation),
    validation_steps=len(X_validation)//batch_size)


Model loaded.
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128

In [ ]:
# evaluate model on test set
data = []
labels = []
DATADIR = "/content/drive/MyDrive/try1000_model_cough/melspectro_selected/testing/coughing/test_cough/1-53663-A-24.png"

CATEGORIES = ['clapping']
training_data = []
#for category in CATEGORIES:  # do cough and notcough

    #path = os.path.join(DATADIR,category)  # create path to cough and notcough
    ##print(path)
    ##class_num = CATEGORIES.index(category)  # get the classification  

    #for img in tqdm(os.listdir(path)):  # iterate over each image per cough and notcough
try:
    img_array = cv2.imread(os.path.join(DATADIR))  # convert to array
    new_array = cv2.resize(img_array, (224, 224))  # resize to normalize data size
    #image = preprocess_input(new_array)
    training_data.append([new_array, 1])  # add this to our training_data
except Exception as e:  # in the interest in keeping the output clean...
    pass
        
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)
X = np.array(X).reshape(-1, 224, 224, 3)
data = np.array(X, dtype="float32")
labels = np.array(y)
#data = np.array(X, dtype="float32")

# perform one-hot encoding on the labels
#lb = LabelBinarizer()
#labels = lb.fit_transform(labels)
#labels = to_categorical(labels)


print(model.predict(data))
model.save("/content/drive/MyDrive/try1000_model_cough/cough_detector.model", save_format="h5")

[[2.3787955e-04 9.8779571e-01 3.7959213e-03 2.3957789e-03 5.7748309e-03]]
